In [1]:
import numpy as np
import pandas as pd

# Problem Understanding

The National Health Service has determined that decisive action to treat diabetes is necessary. However, diagnosing it takes hours of doctors that are in high demand.<br>

Help them to predict who is diabetic and who is not based on data that non-medical stuff can obtain so that you reduce the number of people who:
- get treated without needing it
- don't get treated when they actually needed it

# Data Understanding

https://data.world/data-society/pima-indians-diabetes-database

In [2]:
df = pd.read_csv('pima-indians-diabetes.csv')
print(df.shape)

df.head()

df['Outcome'].value_counts()

(768, 9)


0    500
1    268
Name: Outcome, dtype: int64

In [3]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


# Data preparation

### Split your X data in train and test datasets
Here is the documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [4]:
# initial split between training set and validation set
from sklearn.model_selection import train_test_split
X_train_f, X_test, y_train_f, Y_test = train_test_split (X, y, test_size = .2, random_state = 45)

print(f"My initial training set is {X_train_f.shape} while my initial test set data is {X_test.shape}")
print(f"My initial dependant variable is {y_train_f.shape} while my test dependant variable is {Y_test.shape}")


# 
# 

My initial training set is (614, 8) while my initial test set data is (154, 8)
My initial dependant variable is (614,) while my test dependant variable is (154,)


### Split your train data in train and validation datasets

In [5]:
# splitting again the initial training data into sub-training data and validation set
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, Y_validation = train_test_split (X_train_f, y_train_f, test_size = .2, random_state = 45)

print(f"My initial training set is {X_train.shape} while my validation test set data is {X_validation.shape}")
print(f"My initial dependant variable is {y_train.shape} while my validation dependant variable is {Y_validation.shape}")
# 
# 
# 

My initial training set is (491, 8) while my validation test set data is (123, 8)
My initial dependant variable is (491,) while my validation dependant variable is (123,)


### Scale the 3 datasets using StandardScaler

In [6]:
from sklearn.preprocessing import StandardScaler 
# intitiating the scaler
ss = StandardScaler()

# fitting and transforming the training,validaton and test set
ss.fit_transform(X_train)
ss.transform(X_validation)
ss.transform(X_test)

ss.fit_transform(X_train)
X_train.shape

(491, 8)

# Modelling and Model Evaluation

### Train a logistic regression model with NO regularisation

In [7]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C = 1000000, solver = 'lbfgs')
log_reg.fit(X_train, y_train)


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1000000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Measure the accuracy and ROC_AUC of your model
Here is the documentation: https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics

In [8]:
from sklearn.metrics import accuracy_score, roc_auc_score

# calculating predicted values for my logistic regression
y_predicted = log_reg.predict(X_train)

# calculating the probability of someone being positive based on our model and making it as a single np array
train_prob = log_reg.predict_proba(X_train)[:,1]

# actually calculating accuracy and roc_auc
accuracy_scoring = accuracy_score(y_train,y_predicted)
roc_scoring = roc_auc_score(y_train,train_prob)

print(f'My accuracy score for the training dataset iss {accuracy_scoring.round(2)}')
print(f'My ROC_AUC score for the training dataset iss {roc_scoring.round(2)}')




My accuracy score for the training dataset iss 0.77
My ROC_AUC score for the training dataset iss 0.86


### Train a logistic regression model with L1 regularisation

In [9]:
# applying a l1 regularisation 
log_lasso = LogisticRegression(penalty = 'l1', solver = 'liblinear')
log_lasso.fit(X_train, y_train)



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

### Measure the accuracy and ROC_AUC of your model


In [10]:
# # calculating predicted values for my logistic regression
y_predicted_l = log_lasso.predict(X_train)

# calculating the probability of someone being positive based on our model and making it as a single np array
train_prob_l = log_lasso.predict_proba(X_train)[:,1]

# actually calculating accuracy and roc_auc
accuracy_scoring_l = accuracy_score(y_train,y_predicted_l)
roc_scoring_l = roc_auc_score(y_train,train_prob_l)

print(f'My lasso model accuracy score for the training dataset iss {accuracy_scoring_l.round(2)}')
print(f'My lasso ROC_AUC score for the training dataset iss {roc_scoring_l.round(2)}')


My lasso model accuracy score for the training dataset iss 0.77
My lasso ROC_AUC score for the training dataset iss 0.86


## Which model did you choose? Explain your reasoning

I would choose a model with higher ROC and less accuracy. **In this case L1 Lasso**.
Accuracy is just the measure of how many times we guessed right against the whole population.
However, it doesn't take into account the false negative aspect of counting which in this situation is important as we would miss out on people with diabetes.

# Interprete your winning model

### Extract your intercept and coefficients

In [11]:
coefficients  = log_lasso.coef_
intercept = log_lasso.intercept_

print(f' My coefficients are {coefficients}, Intercept is {intercept}')

 My coefficients are [[ 0.06819506  0.03774622 -0.02272611  0.01467206 -0.00222092  0.09063562
   0.64314139  0.03156812]], Intercept is [-8.39510144]


### Show the difference in probabilities when changing the value of one of your predictors

In [12]:
# initial probability for patients calculated by the lasso model

X_train['Insulin'] = X_train['Insulin'] *5
train_prob_l_2 = log_lasso.predict_proba(X_train)[:,1]

# differences anywhere there was some data available for Insulin
print(train_prob_l.round(2) - train_prob_l_2.round(2))

# changing the dataset back to original data
X_train['Insulin'] = X_train['Insulin'] /5



# 
# 
# 
# 

[0.   0.   0.   0.   0.   0.   0.42 0.23 0.14 0.   0.07 0.23 0.   0.02
 0.28 0.43 0.   0.   0.   0.   0.   0.13 0.   0.   0.18 0.   0.04 0.13
 0.   0.24 0.   0.   0.   0.   0.17 0.   0.   0.   0.08 0.05 0.21 0.05
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.08 0.38 0.13 0.12 0.22
 0.   0.04 0.27 0.   0.   0.48 0.42 0.   0.   0.04 0.22 0.05 0.   0.47
 0.19 0.   0.   0.25 0.   0.36 0.9  0.   0.   0.   0.13 0.42 0.07 0.16
 0.02 0.   0.06 0.15 0.48 0.   0.16 0.   0.   0.48 0.   0.   0.07 0.
 0.   0.   0.22 0.04 0.06 0.   0.   0.11 0.   0.43 0.   0.19 0.07 0.
 0.54 0.08 0.   0.   0.18 0.   0.05 0.   0.   0.01 0.   0.   0.   0.
 0.   0.14 0.09 0.16 0.06 0.   0.36 0.35 0.   0.   0.07 0.28 0.   0.
 0.   0.27 0.14 0.   0.1  0.   0.   0.   0.23 0.   0.   0.17 0.01 0.
 0.   0.01 0.02 0.   0.21 0.23 0.   0.12 0.36 0.   0.1  0.   0.   0.
 0.1  0.   0.   0.   0.01 0.23 0.29 0.   0.18 0.21 0.   0.   0.03 0.27
 0.25 0.29 0.   0.   0.36 0.21 0.02 0.   0.28 0.   0.   0.   0.   0.04
 0.02 0.   0.   0.

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Extract the probability of a positive for your validation samples

In [13]:
# still using the lasso logistic regression
log_lasso_validation = LogisticRegression(penalty = 'l1', solver= 'liblinear' )
log_lasso_validation.fit(X_validation, Y_validation )

# calculating probability for the validation set
validation_positive_prob =  log_lasso_validation.predict_proba(X_validation)[:,1]
validation_positive_prob

# 
# 

array([0.272461  , 0.21215956, 0.09967521, 0.09903868, 0.05292768,
       0.46663825, 0.58648439, 0.33781434, 0.05701316, 0.47153603,
       0.06790356, 0.54018931, 0.25009996, 0.43990249, 0.77075351,
       0.0714337 , 0.10226516, 0.26797398, 0.042863  , 0.62247493,
       0.19330454, 0.05211103, 0.24029727, 0.07938276, 0.15049673,
       0.17461892, 0.26714813, 0.21026371, 0.1026948 , 0.00372388,
       0.37247738, 0.24992262, 0.0389421 , 0.20274177, 0.86309256,
       0.02423544, 0.1619096 , 0.33728958, 0.1026319 , 0.08095139,
       0.1576813 , 0.12282049, 0.56750827, 0.17421436, 0.0860804 ,
       0.34084853, 0.2419918 , 0.04359395, 0.17176606, 0.14814092,
       0.32519829, 0.57038807, 0.44795322, 0.34418954, 0.35450499,
       0.0800847 , 0.09746833, 0.08434813, 0.0295791 , 0.23965582,
       0.58311658, 0.35059387, 0.10169312, 0.67213135, 0.07307416,
       0.32870329, 0.62809424, 0.3938755 , 0.64469841, 0.07156889,
       0.11099194, 0.00583028, 0.68774665, 0.14725764, 0.09382

### Extract FPR, TPR and thresholds

In [14]:
from sklearn.metrics import  roc_curve

# function to count all instances 

def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP+=1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP +=1
        if y_actual[i]==y_hat[i]==0:
            TN+=1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN+=1
    print('Total True Positive are ', TP) 
    print('Total False Positive are ', FP)   
    print('Total True Negatives are ', TN)   
    print('Total False Negatives are ', FN)   

    return(TP, FP, TN, FN)


# making the initial y_train series into an array
y_train_array = np.array(y_train)

# calculating the false/true/negative/positive outcomes on the train lasso logistic regression
perf_measure(y_train_array, y_predicted_l)

TP = 72
FP = 25
TN = 281
FN = 113

fpr = FP / (FP + TN)
tpr = TP / (TP + FN)

# assuming the cut off point for the predict() is 50%!!!
print ('The False Positive rate for the training data at a 50% threshold is ', round (fpr, 2))
print ('The True Positive rate for the training data at a 50% threshold is ', round  (tpr, 2))





Total True Positive are  113
Total False Positive are  40
Total True Negatives are  266
Total False Negatives are  72
The False Positive rate for the training data at a 50% threshold is  0.08
The True Positive rate for the training data at a 50% threshold is  0.39


### Plot the ROC curves for your training, validation (and, if you are done, test) datasets

In [15]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification

fpr, tpr, threshold = metrics.roc_curve(y_train, train_prob_l)
roc_auc = metrics.auc(fpr, tpr)

# training dataset
import matplotlib.pyplot as plt
plt.title('ROC Training Set')
plt.plot(fpr, tpr, 'b', label = 'AUC')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


# # validation dataset 

validation_prob_l = log_lasso.predict_proba(X_validation)[:,1]
fpr_v, tpr_v, threshold_v = metrics.roc_curve(Y_validation, validation_prob_l)
roc_auc = metrics.auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.title('ROC Validation')
plt.plot(fpr_v, tpr_v, 'b', label = 'AUC')
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()



<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

# Threshold selection

### Estimate the prevalence in the environment where your model will be used

In [16]:

# prevalence is equal to all the positives over the total sample

total_positives =  log_lasso.predict(X_train)
mini_df = pd.DataFrame(data = total_positives, columns = ['positive'])

all_positive_on_training_set = mini_df['positive'].value_counts()[1]

# is the threshold for .predict() function .5???
print(f'All positives according to the predict function threshold are {round(all_positive_on_training_set, 2)}')

prevalence = all_positive_on_training_set / len(X_train)

print(f'Prevalence in this environment would be {round(prevalence, 2)}')


All positives according to the predict function threshold are 153
Prevalence in this environment would be 0.31


### Estimate the costs for each unit of your FPs, TNs, FNs and TPs
**Hint:** You don't have data for this. You will have to do some research and deep thinking<br>
**Hint 2:** think of a £ value or something else that non-technical stakeholders could relate to<br>
**Hint 3:** They are going to be approximations and that's fine. Just create a good logic.<br>

FPc = The cost of False Positives in this case is mostly unnecessary time and resources to analyse and help people who don't need help.
This can create longer waiting times along with inefficiencies in terms of medical supplies and material. 

annual = equipment_test(£50000) + doctors_time(?) £200,000 + ongoing_unecessary_treatments £300,0000

TNc =  There's no cost associated with a True statement here.

FNc = This would be mostly a social cost as you'd end up not recognizing the disease when someone is actually sick.
There's no imminent money loss for the medical facilities but society as a whole would experience health issues as most people with diabetes do not get cured in time.
Long-run term can lead to even greater losses both from a human and economic perspective.

Loss of lives(???) £10,000,000 (can have loads of repercussions but hard to quantify it).


TPc =  Ongoing_treatments £300,000



### Calculate your m parameter

In [17]:
CFP = 50000 + 20000 + 300000
TNC = 0
FNC = 10000000
TPC = 300000

m = ((1 - prevalence) / prevalence ) * ((CFP - TNC) / (FNC - TPC))

print( f' m coefficient is {round(m,2)}')



 m coefficient is 0.08


### Calculate fm for each threshold

In [18]:
fpr, tpr, threshold = metrics.roc_curve(y_train, train_prob_l)
roc_auc = metrics.auc(fpr, tpr)

# finding TPR and FPR for threshold 0.4
thresh_40 = threshold[65]
fpr_40 = fpr[65]
tpr_40 = tpr[65]

fm_40 = tpr_40 - m*fpr_40

print(f' The fm for the .4 threshold is {round(fm_40,2)}')

# finding TPR and FPR for threshold 0.5
thresh_50 = threshold[46]
fpr_50 = fpr[46]
tpr_50 = tpr[46]

fm_50 = tpr_50 - m*fpr_50

print(f' The fm for the .5 threshold is {round(fm_50,2)}')

# finding TPR and FPR for threshold 0.6
thresh_60 = threshold[33]
fpr_60 = fpr[33]
tpr_60 = tpr[33]

fm_60 = tpr_60 - m*fpr_60

print(f' The fm for the .6 threshold is {round(fm_60,2)}')

# finding TPR and FPR for threshold 0.7
thresh_70 = threshold[25]
fpr_70 = fpr[25]
tpr_70 = tpr[25]

fm_70 = tpr_70 - m*fpr_70

print(f' The fm for the .7 threshold is {round(fm_70,2)}')



 The fm for the .4 threshold is 0.7
 The fm for the .5 threshold is 0.58
 The fm for the .6 threshold is 0.52
 The fm for the .7 threshold is 0.41


### Select the threshold with the highest fm score

In [19]:
print(f' The the highest fm is the .4 threshold with {round(fm_40,2)}')

 The the highest fm is the .4 threshold with 0.7


### Plot the confusion matrix for your selected threshold

In [20]:

# selecting all patients with above .4 from the predict_proba function on the training set
another_mini_df = pd.DataFrame(data = train_prob_l, columns =  ['probabilities'])

# created another colmns with all the .4 instances
another_mini_df['prob>4'] = (another_mini_df['probabilities'] >=.4).astype(int)

# conveeting into an array
y_pred_4 =  np.array(another_mini_df['prob>4'])

# creating the confusion matrix with sklearn
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred_4)



array([[246,  60],
       [ 50, 135]])

### Calculate your alpha / power / precision / accuracy
(Don't use any library for this exercise, in real life you can though)

In [21]:
# alpha is equal to false positive rate

alpha = 60  / (60 + 135)

# statistical power is basically the true positive rate
stat_power = 246 / (246 + 50)

# precision are the actual positive within all the people I estimated to be positive

prec = 246 / (246 + 60)

# accuracy is the sum of all the true positive and negative over the entire population

accu = (246 + 135) /491

print ('Alpha is ', round(alpha,2))
print ('Statistical Power is ', round(stat_power,2))
print ('Precision is ', round(prec,2))
print ('Accuracy is ', round(accu,2))






Alpha is  0.31
Statistical Power is  0.83
Precision is  0.8
Accuracy is  0.78


### Explain in non-technical terms your alpha / power / precision / accuracy

Alpha is the probability of stating that someone needs treatments while actually they didn't.
Statistical power is how much our model is good at identifying people who actually need some treatments for diabetes.

Precision here would be how many times our model correctly predicted someone needed tratement when the model thought so.
Accuracy would be how many times the model managed to identify correctly both people that needed treatment and people who didn't.

# Actionable problem solving recommendations

### Write a brief paragraph telling your stakeholders something they can **DO** to be better off based on your model

Really not sure on how to interpret the coefficients of the logistic regression and also how to take actionable/meaningful insights from the ROC.
By looking at the coefficients of the lasso model, the variable 'DiabetesPedigreeFunction' seemed to have a much greater impact than anything else.

I would advise doctors to closely monitor that value but it doesn't sound really useful to be honest.